In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
current_pwd = os.getcwd()

possible_paths = [
    '/home/export/soheuny/SRFinder/soheun/notebooks', 
    '/home/soheuny/HH4bsim/soheun/notebooks'
]
    
assert os.getcwd() in possible_paths, f"Did you change the path? It should be one of {possible_paths}"
os.chdir("..")

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import torch

from plots import hist_events_by_labels
from events_data import EventsData
from fvt_classifier import FvTClassifier
from tst_info import TSTInfo
# import LogNorm
from matplotlib.colors import LogNorm


features = [
    "sym_Jet0_pt", "sym_Jet1_pt", "sym_Jet2_pt", "sym_Jet3_pt",
    "sym_Jet0_eta", "sym_Jet1_eta", "sym_Jet2_eta", "sym_Jet3_eta",
    "sym_Jet0_phi", "sym_Jet1_phi", "sym_Jet2_phi", "sym_Jet3_phi",  
    "sym_Jet0_m", "sym_Jet1_m", "sym_Jet2_m", "sym_Jet3_m",
]

In [12]:
# from dataset import MotherSamples
# import tqdm

# for sr in [0.01, 0.02]:
#     for seed in tqdm.tqdm(range(100)):
#         hparams = {
#             "n_3b": 100_0000,
#             "ratio_4b": 0.5,
#             "signal_ratio": sr,
#             "signal_filename": "HH4b_picoAOD.h5",
#             "seed": seed,
#         }

#         ms = MotherSamples.from_hparams(hparams)
#         ms.save(verbose=False)

# MotherSamples.update_metadata()

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [03:37<00:00,  2.18s/it]
300it [00:02, 100.43it/s]


In [9]:
from dataset import MotherSamples

hparams = {
        "n_3b": 100_0000,
        "ratio_4b": 0.5,
        "signal_ratio": 0.0,
        "signal_filename": "HH4b_picoAOD.h5",
        "seed": 0,
    }

ms1 = MotherSamples.from_hparams(hparams)
ms2 = MotherSamples.from_hparams(hparams)

In [10]:
ms1.scdinfo.fetch_data()

,Jet0_pt,Jet1_pt,Jet2_pt,Jet3_pt,Jet0_eta,Jet1_eta,Jet2_eta,Jet3_eta,Jet0_phi,Jet1_phi,...,sym_Jet3_phi,sym_Jet0_phi,sym_Jet0_pt,sym_Jet0_m,sym_Jet1_pt,sym_Jet1_m,sym_Jet2_pt,sym_Jet2_m,sym_Jet3_pt,sym_Jet3_m
0,108.895241,71.013512,51.835068,46.618690,-1.658917,1.977475,-1.210238,-2.287511,-1.494194,-2.696652,...,-2.846889,0,108.895241,0.0,71.013512,0.0,51.835068,0.0,46.618690,0.0
1,227.283508,111.562592,72.964806,43.811436,-1.996783,-1.082387,-1.683465,0.210801,-2.821685,-0.019134,...,2.801995,0,227.283508,0.0,111.562592,0.0,72.964806,0.0,43.811436,0.0
2,390.176117,178.459351,70.963318,53.994999,-1.838448,-0.392330,0.262510,-1.916002,2.657007,-0.545456,...,2.120158,0,390.176117,0.0,178.459351,0.0,70.963318,0.0,53.994999,0.0
3,116.667885,91.205055,71.950462,53.511410,1.564536,-0.619623,0.180431,0.600385,0.476113,2.668455,...,-0.742374,0,116.667885,0.0,91.205055,0.0,71.950462,0.0,53.511410,0.0
4,147.147125,141.689362,141.523926,52.624500,2.177654,0.947881,1.265971,0.085411,-3.097885,-1.361109,...,-0.881319,0,147.147125,0.0,141.689362,0.0,141.523926,0.0,52.624500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094678,123.115860,110.293518,75.215515,43.902065,-0.387770,1.575181,1.094460,2.132855,-0.145979,-1.859997,...,2.883935,0,123.115860,0.0,110.293518,0.0,75.215515,0.0,43.902065,0.0
2094679,111.572647,48.760033,45.804516,42.760387,-2.051495,1.052830,0.680842,2.425722,2.072376,0.300532,...,-2.409347,0,111.572647,0.0,48.760033,0.0,45.804516,0.0,42.760387,0.0
2094680,68.653275,62.651684,60.838261,41.110283,-1.686871,0.915442,-1.496263,-0.799012,-1.846007,1.353166,...,1.889827,0,68.653275,0.0,62.651684,0.0,60.838261,0.0,41.110283,0.0
2094681,119.292236,103.201645,45.786457,42.378479,1.605550,0.279862,1.124532,0.639245,1.139656,-0.084389,...,-2.741812,0,119.292236,0.0,103.201645,0.0,45.786457,0.0,42.378479,0.0


In [11]:
ms2.scdinfo.fetch_data()

,Jet0_pt,Jet1_pt,Jet2_pt,Jet3_pt,Jet0_eta,Jet1_eta,Jet2_eta,Jet3_eta,Jet0_phi,Jet1_phi,...,sym_Jet3_phi,sym_Jet0_phi,sym_Jet0_pt,sym_Jet0_m,sym_Jet1_pt,sym_Jet1_m,sym_Jet2_pt,sym_Jet2_m,sym_Jet3_pt,sym_Jet3_m
0,108.895241,71.013512,51.835068,46.618690,-1.658917,1.977475,-1.210238,-2.287511,-1.494194,-2.696652,...,-2.846889,0,108.895241,0.0,71.013512,0.0,51.835068,0.0,46.618690,0.0
1,227.283508,111.562592,72.964806,43.811436,-1.996783,-1.082387,-1.683465,0.210801,-2.821685,-0.019134,...,2.801995,0,227.283508,0.0,111.562592,0.0,72.964806,0.0,43.811436,0.0
2,390.176117,178.459351,70.963318,53.994999,-1.838448,-0.392330,0.262510,-1.916002,2.657007,-0.545456,...,2.120158,0,390.176117,0.0,178.459351,0.0,70.963318,0.0,53.994999,0.0
3,116.667885,91.205055,71.950462,53.511410,1.564536,-0.619623,0.180431,0.600385,0.476113,2.668455,...,-0.742374,0,116.667885,0.0,91.205055,0.0,71.950462,0.0,53.511410,0.0
4,147.147125,141.689362,141.523926,52.624500,2.177654,0.947881,1.265971,0.085411,-3.097885,-1.361109,...,-0.881319,0,147.147125,0.0,141.689362,0.0,141.523926,0.0,52.624500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094678,123.115860,110.293518,75.215515,43.902065,-0.387770,1.575181,1.094460,2.132855,-0.145979,-1.859997,...,2.883935,0,123.115860,0.0,110.293518,0.0,75.215515,0.0,43.902065,0.0
2094679,111.572647,48.760033,45.804516,42.760387,-2.051495,1.052830,0.680842,2.425722,2.072376,0.300532,...,-2.409347,0,111.572647,0.0,48.760033,0.0,45.804516,0.0,42.760387,0.0
2094680,68.653275,62.651684,60.838261,41.110283,-1.686871,0.915442,-1.496263,-0.799012,-1.846007,1.353166,...,1.889827,0,68.653275,0.0,62.651684,0.0,60.838261,0.0,41.110283,0.0
2094681,119.292236,103.201645,45.786457,42.378479,1.605550,0.279862,1.124532,0.639245,1.139656,-0.084389,...,-2.741812,0,119.292236,0.0,103.201645,0.0,45.786457,0.0,42.378479,0.0
